# No Clustering 

In [1]:
# Read data
import pickle
def load_file(filename):
    with open(filename, 'rb') as fi:
        data = pickle.load(fi)
    return data

In [ ]:
from Data_Preprocessing import general as dp
from Data_Preprocessing import calculate_y as cy
from Data_Preprocessing import Train_test

filename = 'C:/Users/USER/Desktop/AIML/資料/Data'
data = load_file(filename)

data = data[data.isna()['UNIT_POST_CODE'] == False]
data = data[data.isna()['CITY'] == False]
data = data[data.isna()['DISTRICT'] == False]
data = data[data.isna()['LOCATION_ID'] == False]

# Training
data = dp.fillnamean(data)  
data = dp.drop_isable(data)
data = dp.drop_repeatcolumn(data)
data = dp.encoding_onehot(data)
data['START_DATE'] = dp.date2months(data['START_DATE'], 1950)
# data = data.drop(['START_DATE'], axis = 1)
data_train, data_test = dp.split_traintest(data, 0.2)
data_X, data_y = dp.separate_Xy(data_train)
data_Xtest, data_ytest = dp.separate_Xy(data_test)
# outlier_pred, data_X, data_y = dp.outlier_Isolation(data_X, data_y)#another outlier method
outlier_pred, data_X, data_y = dp.outlier_std(data_X, data_y, 4)
data_y = cy.calculate_ymean(data_y)
data_ytest = cy.calculate_ymean(data_ytest)

In [4]:
data_y

,life_insurance,property_insurance,y_mean
8280,1.437819e+06,133133.583333,785476.354167
17360,1.114148e+06,38206.750000,576177.125000
974,1.522667e+05,453.166667,76359.908536
2473,1.140450e+04,6181.166667,8792.833333
18729,6.439803e+05,171077.333333,407528.833333
...,...,...,...
8347,1.931400e+05,71002.000000,132071.000000
13626,9.938236e+05,46904.166667,520363.875000
12208,1.627825e+04,328794.500000,172536.375000
17371,1.755106e+05,48801.166667,112155.875000


In [4]:
for i in data:
    print(i)

START_DATE
AGENT_AGE
ON_BOARD_AGE
SENIORITY
AGENT_SEX
UNIT_POST_CODE
LOCATION_ID
login_mean
contact_mean
contact_custmean
visit_mean
visit_custmean
client_mean
memo_mean
qnr_mean
qnr_custmean
consult_mean
consult_custmean
life_insurance
property_insurance
AGENT_TITLE_AM
AGENT_TITLE_ARM
AGENT_TITLE_AVP
AGENT_TITLE_CA
AGENT_TITLE_CDM
AGENT_TITLE_CMD
AGENT_TITLE_CMM
AGENT_TITLE_CMP
AGENT_TITLE_CMS
AGENT_TITLE_CS
AGENT_TITLE_CSM
AGENT_TITLE_CSS
AGENT_TITLE_DM
AGENT_TITLE_MAM
AGENT_TITLE_MM
AGENT_TITLE_MS
AGENT_TITLE_NCT
AGENT_TITLE_SMM
AGENT_TITLE_SP
AGENT_TITLE_UM
AGENT_TITLE_VRM
CITY_南投縣
CITY_台中市
CITY_台北市
CITY_台南市
CITY_台東縣
CITY_嘉義市
CITY_基隆市
CITY_宜蘭縣
CITY_屏東縣
CITY_彰化縣
CITY_新北市
CITY_新竹市
CITY_新竹縣
CITY_桃園市
CITY_澎湖縣
CITY_花蓮縣
CITY_苗栗縣
CITY_金門縣
CITY_雲林縣
CITY_高雄市


In [5]:
data_X = data_X.iloc[:, :18]
data_Xtest = data_Xtest.iloc[:, :18]

In [4]:
# Cross validation
data_Xc, data_yc = dp.separate_Xy(data)
cy.calculate_ymean(data_yc)
data_yc

,life_insurance,property_insurance,y_mean
0,152266.650406,46349.38881,99308.019608
1,448329.750000,46349.38881,247339.569405
2,54899.583333,46349.38881,50624.486072
3,152266.650406,46349.38881,99308.019608
4,152266.650406,46349.38881,99308.019608
...,...,...,...
19764,152266.650406,4765.00000,78515.825203
19765,152266.650406,46349.38881,99308.019608
19766,9280.083333,61874.00000,35577.041667
19767,152266.650406,5115.75000,78691.200203


# Decision tree

In [18]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import numpy as np


# criterion: {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}
def train_decistiontree(train_X, test_X, train_y, test_y, ml):
    model = DecisionTreeRegressor(criterion='squared_error', 
        min_samples_leaf=19, splitter='best', random_state=5)
    model.fit(train_X, train_y)
    #y_predtrain = model.predict(train_X) 
    #loss_train = Train_test.loss_f(y_predtrain, train_y)
    y_pred = model.predict(test_X)    
    loss = Train_test.loss_f(y_pred, test_y)
    return loss, model

loss_good = 1000000000000
for ml in range(2, 20):
    loss, model = train_decistiontree(data_X, data_Xtest, data_y['y_mean'], data_ytest['y_mean'], ml)
    
    if loss<loss_good:
        loss_good = loss
        best_ml = ml
        best_model = model
print(loss_good, best_ml)  


84958.63749521006 19


In [19]:
best_model

DecisionTreeRegressor(min_samples_leaf=19, random_state=5)

In [8]:
# Cross validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(criterion='squared_error', min_samples_leaf=50, splitter='best', random_state=5)
scores = cross_val_score(model, data_Xc, data_yc['y_mean'], cv=10, scoring='neg_mean_absolute_error')
# metrics.SCORERS.keys()
scores.mean()

-62830.8025247267

# Random forest

In [8]:
from sklearn.ensemble import RandomForestRegressor
from Data_Preprocessing import Train_test
import pandas as pd
import numpy as np

# criterion: {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}
rnd_clf = RandomForestRegressor(n_estimators=10, max_leaf_nodes=20,# oob_score = True
    n_jobs=-1, random_state=42, criterion='poisson')
rnd_clf.fit(data_X, data_y['y_mean'])
y_pred_rf = rnd_clf.predict(data_Xtest)
loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])
loss

83707.23991952039

In [ ]:
loss_good = 100000
for max_leaf in range(2, 30): # max_depth?
    for n_estimators in range(10, 50):
        print('-----------')
        rnd_clf = RandomForestRegressor(n_estimators=n_estimators, max_leaf_nodes=max_leaf,
                                        n_jobs=-1, random_state=42, criterion='poisson')
        rnd_clf.fit(data_X, data_y['y_mean'])
        y_pred_rf = rnd_clf.predict(data_Xtest)
        loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])
        if loss<loss_good:
            loss_good = loss     
            ne_good = n_estimators 
            ml_good = max_leaf

rnd_clf = RandomForestRegressor(n_estimators=ne_good, max_leaf_nodes=ml_good,
                                n_jobs=-1, random_state=42)
rnd_clf.fit(data_X, data_y['y_mean'])
y_pred_rf = rnd_clf.predict(data_Xtest)
loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])
for score, name in zip(rnd_clf.feature_importances_, data_X):
    print(round(score, 2), name)


# Gradient boosting

In [10]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
loss_good = 100000
# # criterion: {“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}
def train_gbrt(data_X, data_y, data_Xtest, data_ytest, lr, max_depth, n_estimators):   
    gbrt = GradientBoostingRegressor(max_depth=max_depth, n_estimators=n_estimators,
                                    learning_rate=lr, random_state=42, criterion='squared_error', n_iter_no_change=10)
    gbrt.fit(data_X, data_y)
    y_pred_rf = gbrt.predict(data_Xtest)
    loss = Train_test.loss_f(y_pred_rf, data_ytest)
    return loss, gbrt


loss_old = 100000
for max_depth in range(2, 30):    
    for n_estimators in range(2, 10):
        loss_good = 100000
        lr = 0.1
        while True:
            loss, gbrt = train_gbrt(data_X, data_y['y_mean'], data_Xtest, data_ytest['y_mean'], lr, max_depth, n_estimators)
            print(f'md={max_depth}, lr={lr}, ne={n_estimators}, loss={loss}')
            if loss>=loss_good:      
                lr = lr*0.8
                if lr <= 0.01:
                    if loss_good<loss_old:
                        gbrt_good = GradientBoostingRegressor(max_depth=md_good, n_estimators=ne_good,
                            learning_rate=lr, random_state=42)
                    break
            else:
                # lr = lr*1.1
                lr_good = lr
                md_good = max_depth
                ne_good = n_estimators
                loss_good = loss     
gbrt_good.fit(data_X, data_y['y_mean'])
y_pred_rf = gbrt_good.predict(data_Xtest)
loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])

In [26]:
gbrt_good

NameError: name 'gbrt_good' is not defined

In [35]:
from sklearn.metrics import mean_squared_error
gbrt_best = GradientBoostingRegressor(
    learning_rate=2.1040543606193775e-10, max_depth=29,
    n_estimators=9, random_state=42, n_iter_no_change=10) # n_iter_no_change??
gbrt_best.fit(data_X, data_y['y_mean'])
y_pred_rf = gbrt_best.predict(data_Xtest)
loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])
#loss = mean_squared_error(y_pred_rf, data_ytest['y_mean'])
loss

58961.818983006866

# HistGradientBoostingRegressor

In [12]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline

hgb_reg = HistGradientBoostingRegressor(random_state=42) # categorical_features??
hgb_reg.fit(data_X, data_y['y_mean'])
y_pred_rf = hgb_reg.predict(data_Xtest)
loss = Train_test.loss_f(y_pred_rf, data_ytest['y_mean'])
loss

83970.82274431223

# Stacking

In [17]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

svm_reg = make_pipeline(StandardScaler(), LinearSVR(epsilon = 0.5, random_state = 42))
svm_reg.fit(data_X, data_y['y_mean'])
y_pred = svm_reg.predict(data_Xtest)
loss = Train_test.loss_f(y_pred, data_ytest['y_mean'])
loss

90355.12538810044

In [75]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

n_estimators = [
        ('rf', RandomForestRegressor(random_state=42)),
        ('svc', make_pipeline(StandardScaler(), LinearSVR(epsilon = 0.5, random_state = 42))),
        ('gbr', GradientBoostingRegressor(learning_rate=2.1040543606193775e-23, max_depth=29,
                          n_estimators=9, random_state=42)),
        ('hgbr', HistGradientBoostingRegressor(random_state=42))        
    ]
stacking_reg = StackingRegressor(
    estimators=[
        ('rf', RandomForestRegressor(random_state=42)),
        ('svc', make_pipeline(StandardScaler(), LinearSVR(epsilon = 0.5, random_state = 42))),
    ],
    final_estimator=GradientBoostingRegressor(learning_rate=2.1040543606193775e-23, max_depth=29,
                          n_estimators=9, random_state=42),
    cv=5  # number of cross-validation folds # cross validation for other model    
)
stacking_reg.fit(data_X, data_y['y_mean'])
y_pred = stacking_reg.predict(data_Xtest)
loss = Train_test.loss_f(y_pred, data_ytest['y_mean'])
loss

68731.7938196908